In [4]:
!pip install gensim

  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 14.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

**Word2Vector**
-

In [1]:
import gensim.downloader
from gensim.models import Word2Vec

In [2]:
# Download the pre-trained model
model_1= gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
model_2= gensim.downloader.load('glove-wiki-gigaword-300')


In [3]:
# 300-dimensional vector for 'computer' i.e word embedding of 'computer'
vector= model_1['computer']
print(vector.shape)
print(vector)

(300,)
[ 1.07421875e-01 -2.01171875e-01  1.23046875e-01  2.11914062e-01
 -9.13085938e-02  2.16796875e-01 -1.31835938e-01  8.30078125e-02
  2.02148438e-01  4.78515625e-02  3.66210938e-02 -2.45361328e-02
  2.39257812e-02 -1.60156250e-01 -2.61230469e-02  9.71679688e-02
 -6.34765625e-02  1.84570312e-01  1.70898438e-01 -1.63085938e-01
 -1.09375000e-01  1.49414062e-01 -4.65393066e-04  9.61914062e-02
  1.68945312e-01  2.60925293e-03  8.93554688e-02  6.49414062e-02
  3.56445312e-02 -6.93359375e-02 -1.46484375e-01 -1.21093750e-01
 -2.27539062e-01  2.45361328e-02 -1.24511719e-01 -3.18359375e-01
 -2.20703125e-01  1.30859375e-01  3.66210938e-02 -3.63769531e-02
 -1.13281250e-01  1.95312500e-01  9.76562500e-02  1.26953125e-01
  6.59179688e-02  6.93359375e-02  1.02539062e-02  1.75781250e-01
 -1.68945312e-01  1.21307373e-03 -2.98828125e-01 -1.15234375e-01
  5.66406250e-02 -1.77734375e-01 -2.08984375e-01  1.76757812e-01
  2.38037109e-02 -2.57812500e-01 -4.46777344e-02  1.88476562e-01
  5.51757812e-02  

In [4]:
# Get words which are similar to 'king', makes use of embedding word
print("Words similar to 'king':", model_1.most_similar("king", topn=5))

Words similar to 'king': [('kings', 0.7138045430183411), ('queen', 0.6510956883430481), ('monarch', 0.6413194537162781), ('crown_prince', 0.6204220056533813), ('prince', 0.6159993410110474)]


In [5]:
# See the linear property of Embedded words
# 'king' - 'man' + 'woman' ~ 'queen'
print(model_1.most_similar(positive=['woman', 'king'], negative=['man'], topn=3))

[('queen', 0.7118193507194519), ('monarch', 0.6189674139022827), ('princess', 0.5902431011199951)]


#Sarcasm Detection

In [6]:
import json

f=open("Sarcasm_Headlines_Dataset_v2.json","r")

sentences= []
labels= []
urls= []

for line in f:
    item=json.loads(line)
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

In [7]:
# 1 --> Sarcastic    and 2 -->Non-Sarcastic
print(len(sentences))
print(len(labels))
print(sentences[0])
print(labels[0])

28619
28619
thirtysomething scientists unveil doomsday clock of hair loss
1


In [8]:
import re
import nltk
import numpy as np
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
def pre_processing(sentences):
  # lower casing
  for sentence in sentences:
    sentence= sentence.lower()

  # removing punctuations and stopwords
  for sentence in sentences:
    punctuations= r'[^\w\s]'
    sentence= re.sub(punctuations, '', sentence)

  # remove stopwords
  for sentence in sentences:
    stop_words_set= set(stopwords.words('english'))
    word_dict= sentence.split()
    filtered_text= [word for word in word_dict if word not in stop_words_set]
    sentence= ' '.join(filtered_text)

  return sentences

In [ ]:
def headline_to_vector_W2V(sentences):
    vector_map= []
    for sentence in sentences:
        vec= []
        for word in sentence.split():
            try:
                vec.append(model_1[word])
            except KeyError:
                continue  # Skip words not in the model
        if vec:
            vector_map.append(np.mean(vec, axis=0))
        else:
            # If no words are found, append a zero vector of the correct size
            vector_map.append(np.zeros(model_1.vector_size))

    return vector_map


In [ ]:
def headline_to_vector_GloVe(sentences):
    vector_map= []
    for sentence in sentences:
        vec= []
        for word in sentence.split():
            try:
                vec.append(model_2[word])
            except KeyError:
                continue  # Skip words not in the model
        if vec:
            vector_map.append(np.mean(vec, axis=0))
        else:
            # If no words are found, append a zero vector of the correct size
            vector_map.append(np.zeros(model_2.vector_size))

    return vector_map


In [13]:
sentences= pre_processing(sentences)
x= headline_to_vector_W2V(sentences)

x= np.array(x)
print(x.shape)

(28619, 300)


In [14]:
y =np.array(labels)
print(y.shape)

(28619,)


In [15]:
from sklearn.model_selection import train_test_split as tts
x_train, x_test, train_label, test_label= tts(x, y, test_size=0.2, random_state=42)

In [16]:
print(x_train.shape)
print(x_test.shape)
print(train_label.shape)
print(test_label.shape)

(22895, 300)
(5724, 300)
(22895,)
(5724,)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=50,
    min_samples_split=4,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)
clf.fit(x_train, train_label)

y_pred = clf.predict(x_test)

print(classification_report(test_label, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.80      0.77      2995
           1       0.76      0.69      0.72      2729

    accuracy                           0.75      5724
   macro avg       0.75      0.75      0.75      5724
weighted avg       0.75      0.75      0.75      5724



In [17]:
def predict(sentence, classifier):
  sentences= [sentence]
  sentences= pre_processing(sentences)
  vec= headline_to_vector_W2V(sentences)
  clf= classifier
  pred=clf.predict(vec)
  if pred[0]==1:
    print(f"{sentence} is --> Sarcastic\n")
  else:
    print(f"{sentence} is --> Non-Sarcastic\n")

In [ ]:
# lets test
sentences=["Dooms day clock of death makes me happy", "I would fall for a bald man XOXO", "I like to study", "I will kill u harry, dumbeldor said calmly",
           "Ofcourse i like to get beaten!!"]
for sentence in sentences:
  predict(sentence, clf)

Dooms day clock of death makes me happy is --> Sarcastic

I would fall for a bald man XOXO is --> Sarcastic

I like to study is --> Non-Sarcastic

I will kill u harry, dumbeldor said calmly is --> Non-Sarcastic

Ofcourse i like to get beaten!! is --> Non-Sarcastic



In [24]:
from sklearn.linear_model import LogisticRegression

clf_2= LogisticRegression(max_iter=10000)  # Increase max_iter for convergence
clf_2.fit(x_train, train_label)


LogisticRegression(max_iter=10000)

In [25]:
from sklearn.metrics import classification_report

y_pred = clf_2.predict(x_test)
print(classification_report(test_label, y_pred))


              precision    recall  f1-score   support

           0       0.79      0.79      0.79      2995
           1       0.77      0.77      0.77      2729

    accuracy                           0.78      5724
   macro avg       0.78      0.78      0.78      5724
weighted avg       0.78      0.78      0.78      5724



In [26]:
# lets test
sentences=["Dooms day clock of death makes me happy", "I would fall for a bald man XOXO", "I like to study", "I will kill u harry, dumbeldor said calmly",
           "Ofcourse i like to get beaten!!"]
for sentence in sentences:
  predict(sentence, clf_2)

Dooms day clock of death makes me happy is --> Sarcastic

I would fall for a bald man XOXO is --> Sarcastic

I like to study is --> Sarcastic

I will kill u harry, dumbeldor said calmly is --> Non-Sarcastic

Ofcourse i like to get beaten!! is --> Non-Sarcastic

